<a href="https://colab.research.google.com/github/phyop/220528_Colab/blob/main/220715_WSGI_%E6%9C%8D%E5%8B%99%E5%99%A8_%E6%A1%86%E6%9E%B6_%E8%B7%AF%E7%94%B1%E8%A3%9D%E9%A3%BE%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
《WSGI - Web伺服器閘道器介面》
https://www.udemy.com/course/python-en/learn/lecture/15690800#overview

# WSGI, Web Server Gateway Interface
# 定義了web server 跟python web application 之間溝通的規範
# F5的重新整理很可能用的是緩存，而ctrl+r是強制更新頁面


In [ ]:
《TCP socket套接字 - 本來是面向過程function》

import socket
import multiprocessing


# 傳入client各別的socket服務物件
def service_client(new_socket):
    # 接受client(瀏覽器)發過來的request，也就是http請求
    # 對request分離出需要的資訊、使用try-except處理，比如返回200或404之類的header，以及主體body
    pass
    # 進程變量不共享，所以子進程也要做socket物件.close()
    new_socket.close()


def main():
    # 使用TCP socket的幾大步驟：
    
    # 1. 創建「socket物件」，socket.socket()
    # SOCK_STREAM -> TCP
    # SOCK_DGRAM -> UDP
    tcp_server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # TCP的3次握手4次揮手後，端口資源不會立刻放掉，而是會繼續保存2～4分鍾
    # 如果server掛了又馬上啓動起來，那會被系統提示端口正在被使用，然而實際上根本就是在揮手後的暫時保存狀態
    # 也就是port被斷開，實際上沒接上，但狀態卻是顯示有人鏈接，所以要去做實際的接上就會被警告有人用，陷入鬼打牆的2～4分鍾
    # 所以這時候要去對socket物件，把SO_REUSEADDR設置成1，如下：
    tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    # 2. 去綁定port號，socket物件.bind()
    # 用瀏覽器去打開127.0.0.1:7890
    tcp_server_socket.bind(("", 7890))


    # 3. 使socket物件進入listen狀態，socket物件.listen()
    tcp_server_socket.listen()
    
    while True:
        # 4. 持續的等待client端進行鏈接，socket物件.accept()
        # 鏈接上client之後，會回傳各別client的socket服務物件，以及client的地址
        new_socket, client_addr = tcp_server_socket.accept()
        
        # 5. 持續的爲client進行服務
        # 定義一個client服務的函式service_client
        # 使用multi-process，讓各個client服務獨立執行，不會因爲client1的堵塞，而讓client2無法執行
        # 要傳入多進程、多線程的參數，要以元組的形式傳入
        p = multiprocessing.Process(target=service_client, args=(new_socket,))
        # 很多物件的創建是不會立刻進入執行，所以需要start()、run()等方法去啓動
        p.start()
        
        # 關閉服務的順序，是開啟服務的順序相反
        # 先有tcp_server_socket，才有new_socket。所以關閉時，先關new_socket
        # 新處於待機開放而被忽視管理的狀態，很容易被駭客盜用，所以要確保不使用socket時是關閉的
        # 多進程的變量不共享，所以主進程、子進程都要做socket物件.close()
        new_socket.close()
    
    # 6. 不使用socket物件時，要記得關閉
    tcp_server_socket.close()

if __name__ == "__main__":
    main()


In [ ]:
《TCP socket套接字 - 進階改成面向對象class》

# 面向對象就是有，繼承(類)、封裝(物件)、多態等特性的體現
# 多態，就是同樣一句函式呼叫，只要參數不同，實際執行的就可以是完全不同的方法
# 舉例來說，在class MiniOS中，有一個物件方法os_install_app(self, app)，會去調用app.install()
# 而這樣一句app.install()，當參數app是不同值的時候，例如pycharm和chrome，實際最後調用的方法是截然不同的
# pycharm和chrome這2個物件的類，都是繼承自同一個父類App的子類
# 當參數是pycharm時，因爲在pycharm物件的類找不到insatall()方法，所以往父類App去找
# 所以參數pycharm的「app.install()」，實際執行的是「App中的insatall方法」
# 當參數是chrome時，因爲chrome物件的類有去override覆寫父類中的insatall()方法，所以在自身類中找得到install()方法
# 所以參數chrome的「app.install()」，實際執行的是「Chrome類下的insatall方法」
# 同樣一句app.install()，實際執行的卻是完全不同的方法，這就是多態


# Overloading重載，則是同一個函式的參數個數，或是參數型態，或是參數順序不同，例如type()就有2種完全不同用法
# Overloading可閱讀性低，很容易造成混淆，所以盡量避免


import socket
import multiprocessing


# 1. 面向過程，函數一個個看起來散亂，可讀性差，不知道哪幾個函數的使用關聯性高
# 要增加可讀性與方便利用，就要使用面向對象class
# 2. 定義一個類，把這些通訊相關的方法封裝起來
class WSGIserver:
    def __init__(self):
        # 6. socket物件的創建、綁定port號、變更狀態爲listen，都是只做一次的初始化，所以應該是放在__init__
        
        # 7. 不加self的話，那就是一般的區域變量，無法讓物件在class中的其他地方被使用
        # 8. 把所有的tcp_server_socket，前面都加上self
        self.tcp_server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.tcp_server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.tcp_server_socket.bind(("", 7890))
        self.tcp_server_socket.listen()

    # 11. 方法要記得判斷是不是加上self當首參數
    def service_client(self, new_socket):
        pass
        new_socket.close()


    # 4. 原本的main()，功能其實就是把服務持續性的跑起來，所以也是個必須封裝起來的方法
    # 5. 不確定該是什麼方法時，就先當作最常用的物件方法，加上self
    # 然後去看內部的code，把該加self的地方加一加，看加上去邏輯會不會有問題
    def run_forever(self):                
        while True:
            new_socket, client_addr = self.tcp_server_socket.accept()

            # 9. 方法的話是物件內部所以要加上self，函數是全局才不用加self
            # 10. args要傳遞的參數new_socket，只在這個方法裏面會用到而已，是區域變量，而不是共享的，所以不需要加self
            p = multiprocessing.Process(target=self.service_client, args=(new_socket,))
            p.start()
            
            new_socket.close()

        self.tcp_server_socket.close()



# 3. 經過類的封裝後，這樣的main才會邏輯簡單，而不是原本那麼一長串復雜
def main():
    # 創建通訊類的物件，才可以調用封裝於類的方法
    wsgi_server = WSGIserver()
    wsgi_server.run_forever()


if __name__ == "__main__":
    main()


In [ ]:
《py檔解耦 - 把會常更動的code，遷移出去獨立一個檔》
https://www.udemy.com/course/python-en/learn/lecture/15690818#overview

# 假設把比較不會更動的，比如靜態網頁、http server的處理，寫在web_server.py
# 然後把比較會更動的，比如動態網頁，另外丟到一個mini_frame.py去處理
# 這邊假設，動態網頁以py結尾命名，靜態網頁以其他結尾命名
# 當page_name是動態網頁時，那就同一去調用動態模組mini_frame.py下的application()函式來處理
# 這樣只要處理的動態模組名稱mini_frame、函式名application不變，那web_server.py基本上就不會再需要更動了


# mini_frame.py由一個固定的接收入口application()，來判斷發過來的動態網頁需求
# 然後再分發去合適的函式，比如login()、register()做處理
# 所以間接可以得到一個現象：browser請求不同的頁面，其實對應到的是不同的函數
# 現象2：服務器的通訊，比如指的就是這邊的web_server.py，基本上是不會改變的，會改動的是頁面顯示的內容
# 有名的WSGI服務器（相當於web_server.py）：Nginx、Apache、Waitress、GUnicorn
# 服務器可以支援各種框架（相當於mini_frame.py）：Pyramid、Flask、Django
# WSGI就是定義了，服務器要怎麼去調用框架、參數是怎麼傳，框架應該回給服務器什麼、實際回的格式是怎麼樣，之類的各種協議


# web_server.py
import mini_frame

def page_service(self, new_socket, page_name):
    if not page_name.endswith(".py"): 
        pass
    # 如果發過來的request是動態網頁需求，那就丟去mini_frame.py的application()處理
    else: 
        header = "HTTP/1.1 200 OK\r\n"
        header += "\r\n"
        body = mini_frame.application(page_name)
        response = header + body
        new_socket.send(response.encode("utf-8"))

    new_socket.close()


# mini_frame.py
import time

def application(page):
    # 函式application(dict, page)，處理要調用的頁面外，其實還需要一個包含所有http請求的字典，這樣才不用寫一堆if-else
    if page == "/login.py":
        # 當初request來的時候，是一層層的丟下來到這裏
        # 所以要把做完的結果，也一層層的return回去
        return login()
    elif page == '/register.py':
        return register()
    else:
        retun "not found oh..."

def login():
    # 靜態頁面，除非後端送出的資訊有改變，不然顯示的內容完全不會變化
    # 動態頁面，雖然後端code根本沒改變，但顯示的內容，會隨着ctrl+r重整，而改變
    # 動態比如頁面上的時間顯示，你什麼時候跟我要，我就顯示那個當下的該有的畫面給你
    return "login, time: %s" % time.ctime() 

def register():
    return "register page, time: %s" % time.ctime() 


In [ ]:
《WSGI - 瀏覽器、服務器、框架》
https://www.udemy.com/course/python-en/learn/lecture/15690826#overview

# 1. 瀏覽器發送http請求給web server
# 2. web server發現是動態網頁，自己沒辦法處理，要讓應用框架去處理
# 3. web server會存有一個各種http請求的對應dict，以及發送http響應的func_p，一起通過調用application(dict, func_p)傳給框架
# 4. 框架中的application()中的code，會根據func_p，來決定回什麼header給web server
# 5. application要回給web server的方法就是，調用func_p，並傳入要回覆的header當作實參
# 6. web server的func_p就會得到實參header的值，所以可以把header先保存起來
# 7. web server的application繼續往下做，生成出動態的body，然後通過return，把body傳回去給web server
# 8. web server把通過func_p先拿到的header，再結合application返回的body，傳回給瀏覽器

def application(environ, start_response):
    start_response('200 ok', [('Content-Type', 'text/html'), ..., (cookie名, cookie值)])
    return body


In [ ]:
《WSG範例 - 服務器、框架》

# 因為服務器和框架解耦，也就是理論上可以實現，把服務器換nginx，框架換成Flask


# 服務器 - web_server_v1.0.py
import mini_frame

def page_service(self, new_socket, page_name):
    if not page_name.endswith(".py"): 
        pass
    else: 
        # 要調用框架的application()，所以要先準備好dict
        # 以及func_p，也就是set_response_header()
        env = dict()
        # set_response_header不是全局函數，而是物件方法，所以不加self會找不到
        body = mini_frame.application(env, self.set_response_header)

        # 要先調用application之後，才會有headers可用
        header = "HTTP/1.1 %s\r\n" % self.status
        # 因爲headers是一個list，裏面有很多個元組
        # 而打開google瀏覽器的檢查 -> Network -> Headers -> Response Headers
        # 裏面顯示那些cookie類的都是一個key-value就是一行，所以才會在%s:%s後面，加上\r\n
        for h in self.headers:
            header += "%s:%s\r\n" % (h[0], temp[1])
        # header和body中間有空行
        header += "\r\n"
        response = header + body
        new_socket.send(response.encode("utf-8"))

    new_socket.close()


# application利用調用set_response_header，傳來實參header，所以這邊要有相對應的形參來接收
# 所以這個函數的主要功用，其實就是接收response的header訊息
# 既然如此，那server如果也想提供header，比如server自己的版本號，就也會寫在這裏
# 所以才有了self.headers = [('server', 'web_server_v1.0')]這一行
def set_response_header(self, status, headers):
    # 因爲要傳給page_service()，不能使用區域變量，所以要加上self，讓變量變物件屬性，來永久儲存(直到物件沒了)
    # 和Response Headers同樣階層的General，則會顯示「Status Code : 200 ok」
    self.status = status
    # 這樣去看Response Headers，就可以看到「server : web_server_v1.0」
    self.headers = [('server', 'web_server_v1.0')]
    # 然後再和application傳來的headers用列表連接，點view source看全部的headers時，就也可以看到body傳的Content-Type了
    self.headers += headers
    

# 框架 - mini_frame_v1.0.py
def application(environ, start_response):
    # 如果body有亂碼的話，那就隨便打開一個中文頁面，復制Content-Type的值，text/html;charset=utf-8
    # 右鍵 -> 檢查 -> Response Headers -> Headers -> 「Content-Type : text/html;charset=utf-8」
    start_response('200 ok', [('Content-Type', 'text/html;charset=utf-8')])
    body = 'Hello World!'
    return body


In [ ]:
《WSG範例 - application的dict()參數》

# 服務器 - web_server_v1.0.py
def page_service(self, new_socket, page_name):
    env = dict()
    # 字典env不止包含網址，還可能有server版本、使用的瀏覽器、OS等資訊
    # 而我們寫的框架，就可以因應字典env中的這些資訊，而有不同的作動
    env['PATH_INFO'] = page_name


# 框架 - mini_frame.py
def index():
    return "主頁"

def loging():
    return "登錄頁"

def application(environ, start_response):
    start_response('200 ok', [('Content-Type', 'text/html;charset=utf-8')])
    
    # 從web_service的page_service傳來實參env，給這邊的形參environ
    page_name = environ['PATH_INFO']

    # 然後application就可以取用到網址，做函數的映射
    if page_name == "/index.py":
        return index()
    elif page_name == "/login.py":
        return login()
    else:
        return "not found"


In [ ]:
 《WSGI - web樹》

# 使用tree命令可以看到，基礎的web服務包含4大項目：HTML骨架模版、static裝飾、dynamic框架、服務器
# templates是骨架，下面會有很多html文件
# 對骨架模版的裝飾，會和templates放在同階層的static下面，包含CSS和JS

# 如果要做成package，Python2規定要有__init__.py，Python3則不一定
# 需要有__init__.py的用意是，要在裏面指定「對外界提供」的模組列表，比如「from . import 要對外界提供的模組名」，其中.是當前目錄
# package的__init__.py，定義了這樣的模組列表之後，main file在import package之後，打了package的後面，才會智能提示「有提供的模組名」
# 就可以在pycharm點擊package，然後在頂列選擇「工具」 -> 「創建setup.py」，按照格式填入資訊，就會幫你自動建立了
# 或是使用分发（distribute）工具（utlis），手動創建setup.py

# 既然web_server.py被放到了dynamic資料夾中，那導入就變成了「import dynamic.mini_frame」
# 所以「mini_frame.application()」的code，前面也都要加入「dynamic.」

# $ tree
|-- templates
|    |-- index.html
|    |-- center.html
|-- static
|    |-- css
|        |-- main.css
|        |-- bootstrap.min.css
|    |-- js
|        |-- jquery.css
|        |-- bootstrap.min.js
|-- dynamic
|    |-- __init__.py
|    |-- mini_frame.py
|-- web_server.py


In [ ]:
《WSGI解耦 - 前後端》

# 步驟：
# browser發送request
# server判斷是static就直接回
# dynamic的話，就通過application單一接口丟過去給frame處理
# application會先回header給server的set_response_header
# 然後application會調用對應的方法，返回body，也就是前端寫好的html(template)
# server將header、body做結合，response給browser
# brower解析html檔，發現需要讀取static中的css、js檔，再次發送request給server
# server收到static的request，所以讀取路徑資料檔，再次response給browser
# browser得到所有html、css、js的資料，所以完整解析出user期待看到的頁面


# 服務器 - web_server.py
# 因為mini_frame是框架，所以被移動到dynamic下了
import dynamic.mini_frame

def page_service(self, new_socket, page_name):
    # 靜態的內容和服務器的code都是不太會更動的，所以遇到靜態網頁的話，直接從服務器這邊返回就好了
    if not page_name.endswith(".py"): 
        try:
            f = open("./static + page_name", "rb")
        except:
            pass
    # 動態的話，因為code常常需要更動，所以解耦出去到框架解析
    else: 
        pass


# 框架 - mini_frame.py
def application(environ, start_response):
    start_response('200 ok', [('Content-Type', 'text/html;charset=utf-8')])
    
    # 從web_service的page_service傳來實參env，給這邊的形參environ
    page_name = environ['PATH_INFO']

    # 然後application就可以取用到網址，做函數的映射
    if page_name == "/index.py":
        return index()
    elif page_name == "/center.py":
        return center()
    else:
        return "not found"

def index():
    # file的相對位置，都要從執行的地方，去做相對位置思考
    # 雖然當前文件是mini_frame.py，所以去思考templates/index.html的相對位置時，自然會從當前file檔去相對思考，但是這會出錯！
    # 因為在terminal運行服務的時候，執行的是web_server.py，所以要寫成./templates/index.html
    with open("./templates/index.html") as f:
    return f.read()    

def center():
    # 所以後端寫好服務器、API接口是不太需要改的
    # 而前端去怎麼修改html檔，只要路徑名不變，隨他愛怎麼改就怎麼改，不會影響到後端
    with open("./templates/center.html") as f:
    return f.read()


In [ ]:
《sys.path.append - 临时添加搜索路径》
https://cloud.tencent.com/developer/article/1406466

python程序中使用 import XXX 时，python解析器会在当前目录、已安装和第三方模块中搜索 xxx，如果都搜索不到就会报错。 
 使用sys.path.append()方法可以临时添加搜索路径，方便更简洁的import其他包和模块。这种方法导入的路径会在python程序退出后失效。

1. 加入上层目录和绝对路径
import sys
sys.path.append('..') # 表示导入当前文件的上层目录到搜索路径中
sys.path.append('/home/model') # 绝对路径

2. 加入当前目录
import os,sys
sys.path.append(os.getcwd())
os.getcwd() # 获取当前工作目录

3. 定义搜索优先顺序
# 定义搜索路径的优先顺序，序号从0开始，表示最大优先级，sys.path.insert()加入的也是临时搜索路径，程序退出后失效。
import sys
sys.path.insert(1, "./model")


In [ ]:
《正則 - (..)+?非贪婪模式》

https://blog.csdn.net/LaoYuanPython/article/details/100009688

https://www.runoob.com/python/python-reg-expressions.html

“+”的匹配是贪婪的，在匹配到结果后并不会停止，会继续匹配，直到匹配不到时再回退到上一个匹配位置作为匹配结果，
因此"a1b2c3"匹配时，先后匹配到“a1”、“b2”、“c3”，然后匹配失败，回退到上一次匹配结果“c3”,
因此最后匹配结果是“c3”，如果是非贪婪模式则匹配结果是“a1”。

re.match(pattern, string, flags=0)

>>> m = re.match(r"(..)+", "a1b2c3") # (..)+贪婪模式
>>> m.groups()
('c3',)

>>> m = re.match(r"(..)+?", "a1b2c3") # (..)+?非贪婪模式
>>> m.groups()
('a1',)


In [ ]:
《WSGI解耦 - shell參數、script、readme.txt》

# test.py
import sys
print(sys.argv)

# argv都被當成字串
$ python3 test.py 7890 mini_frame:application
# ['test.py', '7890', 'mini_frame:application']

# 如果可以把port、框架名、調用方法名都從服務器解耦出來，那就自由度更大了
# 例如在啟動服務器的時候，可以指定port號、指定動態請求要調用的框架、方法名


# 每次都要寫一長串shell指令很麻煩，乾脆存成一個script
$ vim run.sh
python3 web_server.py 7890 miniframe: application


# 使用不是那麼直觀的時候，那就最好寫個readme.txt
$ vim readme.txt
運行方法1: ./run.sh
運行方法2: python3 web_server.py 7890 miniframe: application
version: 1.0
date: 2022/07/10

In [ ]:
《WSGI解耦 - port號、框架名》

# a. 即使做了port號、框架名的解耦，但code中關於static、dynamic的位置還是寫死的
# b. 所以應該有一個configuration文件檔，讓user去配置這些偏好
# c. 因爲這只是一個conf檔，而不是py檔，所以不是字典，而是單純的一堆字符串而已
$ vim web_server.conf
{
    "static_path":"./static",
    "dynamic_path":"./dynamic"
}


# web_server.py
import sys

Class WSGIserver:
    def __init__(self, port, application, static_path):
        # 3. WSGIserver的init當然要有形參，才能接收terminal送來的實參，創建物件
        # 所以把原本固定的7890通訊埠，改成形參變量名port
        # 4. 因爲port一旦綁定好了，之後其他地方就沒有需要再使用到port號，所以不需要建一個self.port = port
        self.tcp_server_socket.bind("", port)
        self.application = application
        # h. 方法中需要用到的值，就存成物件屬性
        self.static_path = static_path

    def page_service(self, new_socket, page_name):
        if not page_name.endswith(".py"):
            try:
                f = open(self.static_path + page_name, "rb")
            # 例外發生， 執行except、finally
            except:
                response = "HTTP/1.1 404 not found\r\n" # header
                response += "\r\n" # header和body之間的空行
                response += "--page not found--" #  body
            # 例外沒有發生，執行else、 finally
            else:
                html_content = f.read()
                f.close()
                response = "HTTP/1.1 200 OK\r\n" # header
                response += "\r\n" # header和body之間的空行
                # 發送response header給瀏覽器
                new_socket.send(response.encode("utf-8"))
                # 將mini_frame:application傳回來的body發送給瀏覽器
                new_socket.send(html_content)

        else: 
            env = dict()
            # H. 把原本的「mini_frame.application」，改成「self.application」
            body = self.application(env, self.set_response_header)

    def usage():
        print("usage: $ python3 web_server.py 7890 mini_frame:application")

def main():
    # 1. 爲了讓port號、框架名:調用方法名，可以從code中解耦，所以指令方式或是conf檔方式傳入
    # 這樣就可以做出多個不同權限的框架，讓user根據需求去購買選用
    if len(sys.argv) == 3:
        # 5. 考慮到port可能會有佔用或不存在的可能，所以加入try-exception
        try:
            # 因爲argv都會被解析成字串，所以要自己轉成int
            port = int(sys.argv[1]) # 7890
            frame_str = sys.argv[2] # mini_frame:application
        except Exception as e:
            print("e")
            # 一旦指令有錯誤那就跳出來，所以才加入return來結束這個函數的調用
            return
    else:
        usage()
        return

    # A. 因爲要把框架解耦出來，所以不要使用import mini_frame，這種固定的框架名
    # 「mini_frame:application」，以:爲分隔，分組取
    # 前面是一直匹配到非:的部分，所以^:號
    # 並且:前至少有一個值，不然框架名就是空着的了，所以+號
    # 知道:也都匹配到了之後，剩下的就都是函數名，所以.*號
    frame_application = re.match(r"(^:):(.*)", frame_str)
    # 如果有匹配到的話
    if frame_application:
        frame_str = frame_application.group(1) # mini_frame
        applicaiton_str = frame_application.group(2) # application
    else:
        usage()
        return

    # d. 所以在py檔中，要把conf讀進來，然後轉成dict去取值
    # e. eval()除了可以把看起來是數學運算式但其實是字串，轉成一般計算外，也可以使用於.conf檔轉dict()
    with open("./web_server.conf") as f:
        conf = eval(f.read())  # conf是字典


    # B. 當解析出來框架名後，那就可以做import frame_str
    # 然而，如果照着直接寫，那就會把「frame_str」當作一個字串去尋找module，而不是變量名
    # C.如果要import的是一個變量名，那就必須使用__import__(變量名)
    # 但新的問題是，module的搜尋路徑是当前目录、已安装和第三方模块中，而我們的mini_frame放置的位置，都不在那些地方
    # D. 所以要加入臨時的搜尋路徑到sys.path中
    # 當然，因爲terminal執行的路徑是web_server.py，所以這邊的.指的就是web_server.py那層
    # sys.path.append("./dynamic")
    # f. 所以把固定的路徑，改成變量名
    sys.path.append(conf["dynamic_path"])

    # 本來import模組，是不需要再給一個變量名去指向的，但因爲我們原本寫的是mini_frame:application
    # 所以需要有一個變量名，去指向mini_frame這個module才可以
    frame = __import__(frame_str)
    # 又如果要去一個module定位到函數的位置，可以使用getattr(物件名, 屬性名)
    application = getattr(frame, applicaiton_str)
    # E. frame、application，才是真正指向module、函數位置的變量名
    # frame_str、application_str，只是個變量名字本身的字串而已

    # 2. 要把接受到的指令值，拿來創建WSGIserver物件
    # F. 和port同理，加入frame:applicaiton的實參，來去創建WSGIserver物件
    # 因爲web_server.py和爲了和框架解耦，所以只用到application()當作單一進入口
    # 所以對web_server.py如果只知道applicaiton的位置，是夠用的
    # G. 不需要一定傳入frame的位置給init，因爲其他方法也不會用到
    # g. 因爲的static也要用到，所以也要透過物件的屬性，傳過去
    wsgi_server = WSGIserver(port, application, conf["static_path"])
    wsgi_server.run_forever()


In [ ]:
《將 if-else 改成 dict》

# 很多對應狀況的話，一直用if-else看起來會很不成熟，所以改成用字典去對應

# 框架 - mini_frame.py
# 全局變量，最好用全部大寫，或是至少前面加個global的g_做標示
URL_FUNC_DICT = {
    "/index.py" : index,
    "/center.py" : center
}

def application(environ, start_response):
    start_response('200 ok', [('Content-Type', 'text/html;charset=utf-8')])

    # if page_name == "/index.py":
    #     return index()
    # elif page_name == "/center.py":
    #     return center()
    # else:
    #     return "not found"

    func = URL_FUNC_DICT[file_name]
    return func()
    
def index():
    pass

def center():
    pass
    

In [ ]:
《將 if-else 改成 dict - 無參數裝飾器》

# 總是要有調用原函數的時候嘛，所以inner中一定會去調用原函
# 裝飾器只是可以在調用函數的前後，多增加一些功能
def decrator(func):
    def inner(*args, **kwargs):
        return func(*args, **kwargs)
    return inner


# orifunc = decrator(orifunc)
# 把orifunc當做實參，傳給@decrator的形參func
@decrator
def orifunc():
    pass
    

In [ ]:
《裝飾器 - 步驟複習》

https://colab.research.google.com/drive/1cANpSySUJ8Ie_Wq--TSmTs-SJ6HLYkUE?authuser=1
參考 - 《裝飾器 - 步驟拆解》
參考 - 《裝飾器 - 接收不定參數》

func_list = list()

def decrator(func):
    # 3. 在執行真正的主體inner之前，就會先執行outer的部分
    # 4. 所以每當compiler看到一次@decorator，那outer的這行就會被執行一次
    # 5. 也就是每當有一個函數被掛載@，func_list就會把函數名append進去list中
    # 6. 然後讓那個函數名，指向@decorator的inner
    # 所以當調用函數()，其實就是調用inner()
    # 閉包和裝飾器的不同在於，閉包的使用像是class來創建物件，裝飾器的使用就是對函數增加一些code
    # 閉包類比於class，所以閉包名(outer實參)的調用就像是創建物件，實參就是物件屬性，outter的值就是類屬性
    # 閉包創建的物件名，指向inner，閉包物件名(inner實參)的調用，就像是傳傳遞inner實參給inner這個物件方法
    func_list.append(func)
    def inner(*args, **kwargs):
        return func(*args, **kwargs)
    return inner


# 1. orifunc = decrator(orifunc)
# 2. 一旦看到@，decorator的outter就會被執行，但inner還不會被執行，只是讓orifunc指向而已
@decrator
def orifunc():
    pass


In [ ]:
《裝飾器 - 自動將函數加入list》

# 可以觀察到，函數只要掛上@，這個函數名就會被當作參數，丟進裝飾器去
# 所以我們可以利用這樣的特性，把函數掛上@，經由裝飾器自動加入列表
# 裝飾是只要看到@就會開始啓動outer(裝飾器從內到外打包)，然後讓orifunc函數名指向inner，但不執行inner
# 直到調用函數orifunc()，才會開始執行inner()(從外到內解開裝飾)
# 只要一執行這個py檔，即使orifunc函數都還沒被調用
# 仍會因爲看到@就立刻進行裝飾，而把func這個函數名，加入到func_list中


func_list = list()

def decorator(func):
    func_list.append(func)
    def inner(*args, **kwargs):
        return func(*args, **kwargs)
    return inner


@decorator
def orifunc():
    pass


In [ ]:
《將 if-else 改成 dict - 有參數裝飾器》

# mini_frame.py
# # 自己一個個手動寫字典，麻煩
# URL_FUNC_DICT = {
#     "/index.py" : index,
#     "/center.py" : center
# }

# 同理，如「自動將函數加入list」只要一執行進這個mini_frame.py檔，即使center、index函數都還沒因爲web_server而被調用
# 1. 但仍會因爲進行裝飾，而把各個函數名加入到字典URL_FUNC_DICT
URL_FUNC_DICT = dict()


# 3. 但如果要做成字典，那就必須要有index.py" : index兩種值的對應，而不像加入index只需要一種值
# 4. 既然需要多一個值，那就是要傳入多一個參數給裝飾器，所以做成帶參數型裝飾器，也就是再多包一層def，變3層def
# 5. 取名route，是因為就是個轉址的服務
def route(url):
    def decorator(func):
        # 6. 對每個掛有@route的函數，都自動加進字典對應
        URL_FUNC_DICT[url] = func # "/index.py" : index
        def inner(*args, **kwargs):
            # 7. 在這個case中，對orifunc的調用並不是從inner
            return func(*args, **kwargs)
        return inner
    return decorator


@route(/index.py)
def index():
    pass


@route(/center.py)
def center():
    pass


def application(environ, start_response):
    start_response('200 ok', [('Content-Type', 'text/html;charset=utf-8')])
    page_name = environ['PATH_INFO']

    # if page_name == "/index.py":
    #     return index()
    # elif page_name == "/center.py":
    #     return center()
    # else:
    #     return "not found"

    # 11. 既然是字典，那就會有找不到而發生異常的可能，所以用try-except包起來
    try:
        # 1. 像上面那樣寫一堆if-else做對應看起來很弱，所以把一一對應的情況改用字典做
        # 8. 而是把網址丟去字典做查詢，得到對應的函數
        func = URL_FUNC_DICT[page_name]
        # 2. 直接把對應到的函數，將調用結果回傳
        # 9. 然後在application()這邊做調用
        # 10. 所以，這就是一個沒有利用裝飾器inner去調用，但利用了自動執行outer特性的特殊例子
        return func()
    except Exception as e:
        # 12. 怕e會有特殊格式而在print發生問題，所以統一轉成string
        return "異常訊息：%s" % str(e)
        


# 服務器 - web_server_v1.0.py
def page_service(self, new_socket, page_name):
    env = dict()
    # 字典env不止包含網址，還可能有server版本、使用的瀏覽器、OS等資訊
    env['PATH_INFO'] = page_name
